<a href="https://colab.research.google.com/github/Portman123/Fallout-Hotels/blob/main/Project%20file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import folium
import pandas as pd
import random

# Set up dataframes
nuclear_df = pd.read_csv("https://raw.githubusercontent.com/Portman123/Fallout-Hotels/main/nuclear_df.csv")
nuclear_df = nuclear_df.astype(dtype = {"zone_1": "int64", "zone_2": "int64"})

hotels_df = pd.read_csv("https://raw.githubusercontent.com/Portman123/Fallout-Hotels/main/hotelsdangerzones.csv")
hotels_sample = hotels_df.sample(500)

citydf = pd.read_csv("https://raw.githubusercontent.com/Portman123/Fallout-Hotels/main/worldcities.csv")

# List of random powers
power_list = pd.read_csv("https://raw.githubusercontent.com/Portman123/Fallout-Hotels/main/powersdf.csv")

# Reset index of hotels sample
hotels_sample = hotels_sample.reset_index()

# Making a list of bad super powers for the hotels sample
badsuperpowers = []
for i in range(len(hotels_sample["name"])):
  x = random.randint(0, (len(power_list)-1))
  badsuperpowers.append(power_list.at[x, 'superpower'])

# Add superpowers to the sample df
bsp_df = pd.DataFrame(badsuperpowers)
hotels_sample["superpower"] = bsp_df

# Assign no power to those with 0 danger
mask = hotels_sample["danger-zones"]==0
hotels_sample.loc[mask, "superpower"]="none! there is no radiation here"


In [40]:
def citychoose(pop):#function so executes on page load with new info
  return citydf.loc[citydf['population']>pop]

In [41]:
def mapmaker(plothot, plotplant, plotbomb, pop, bombfallscale, bombboomscale,reactorfallscale, reactorboomscale, mindanger): #function so executes on page load with new info
  m = folium.Map(location=[53.814485, 12.999019], tiles ='Stamen Toner', zoom_start = 4)
  cities = citychoose(pop)#should make the cities contain only those with population > pop
  if(plotplant):
    for lat, lon, name, blast_rad, cont_rad in zip(nuclear_df['latitude'], nuclear_df['longitude'], nuclear_df['name'], nuclear_df['zone_1'], nuclear_df['zone_2']):
        # Creating the marker
        folium.Marker(
        # Coordinate of the country
        location=[lat, lon],
        # The popup that show up if click the marker
        popup=folium.Popup(f"""Nuclear power station - <b>{name}</b> <br>
                            Blast radius: {blast_rad*reactorboomscale} miles <br>
                            Contamination radius: {cont_rad*reactorfallscale} miles <br>
                        """, max_width=len("Nuclear power station - " + name)*10),
        # Change icon
        icon=folium.features.CustomIcon("https://i.ibb.co/bWF2MRf/radiation.png", icon_size=(20, 20))
        ).add_to(m)

    # Zone 2 radius
    for lat, lon, rad in zip(nuclear_df['latitude'], nuclear_df['longitude'], nuclear_df['zone_2']):
      folium.Circle([lat, lon], radius=rad*1609.34*reactorfallscale, color='#ff0000', fill_color='#ff0000', fill_opacity=0.3, opacity=0.3).add_to(m)

    # Zone 1 radius
    for lat, lon, rad in zip(nuclear_df['latitude'], nuclear_df['longitude'], nuclear_df['zone_1']):
      folium.Circle([lat, lon], radius=rad*1609.34*reactorboomscale, color='#fafa0a', fill_color='#fafa0a', fill_opacity=0.6, opacity=0.6).add_to(m)
  if (plothot):
    # Hotels
    for lat, lon, name, superpower, danger in zip(hotels_sample['latitude'], hotels_sample['longitude'], hotels_sample['name'], hotels_sample['superpower'], hotels_sample['danger-zones']):
      if (danger >= mindanger):
        # Creating the marker
        folium.Marker(
        # Coordinate of the country
        location=[lat, lon],
        # The popup that show up if click the marker
        popup = folium.Popup(f"""Hotel name - {name} <br>
                            Number of danger zones: {danger} <br>
                            The power you could get is: {superpower} <br>
                        """, max_width=len(f"{name}")*20),
        # Change icon
        icon=folium.Icon(color="lightgreen")
        ).add_to(m)

  #Cities / Bombs
  if (plotbomb):
    for lat, lon, city, population in zip(cities['lat'], cities['lng'], cities['city'], cities['population']):
      if (population > pop):#Technically redundant, but dont want it to break
        #Creating the marker
        folium.Marker(
        #Coordinate of the country
        location=[lat, lon],
        #The popup that show up if click the marker
        popup=folium.Popup(f"""Major city - <b>{city}</b><br>
                        """, max_width=len(f"Major city - {city}")*10),
        icon=folium.features.CustomIcon("https://i.ibb.co/2sCs96V/Skull-Icon-svg.png", icon_size=(15, 15))
        ).add_to(m)


    for lat, lon, population in zip(cities['lat'], cities['lng'], cities['population']):
      if (population > pop): #Technically redundant, but dont want it to break
        #Fallout radius (based on Castle Bravo)
        folium.Circle([lat, lon], radius=76*1000*bombfallscale, color='#ff0000', fill_color='#ff0000', opacity=0.2).add_to(m)
        #Fireball radius (based on Castle Bravo)
        folium.Circle([lat, lon], radius=7.2*1000*bombboomscale, color='#000000', fill_color='#000000', opacity=0.3).add_to(m)

  return m  

In [42]:
!pip install flask-ngrok

In [43]:
formhtml = '''<form action="/" method = "POST">
<p style="font-size:24px;">Nuclear Bomb Locations</p>
<p>Minimum pop. value for nuke (standard = 1000000) <input type = "text" value="1000000" name = "Minpop" /></p>
<p style="font-size:24px;">Nuclear Bomb Multipliers</p>
<p>Plot Bombs <input type="checkbox" name="PlotBomb" checked></p>
<p>Explosion Scale Value (standard = 1) <input type = "text" value="1" name = "BBoomscale" /></p>
<p>Fallout Scale Value (standard = 1) <input type = "text" value="1" name = "BFallscale" /></p>
<p style="font-size:24px;">Nuclear Power Plant Multipliers</p>
<p>Plot Reactors <input type="checkbox" name="PlotPlant" checked></p>
<p>Explosion Scale Value (standard = 1) <input type = "text" value="1" name = "RBoomscale" /></p>
<p>Fallout Scale Value (standard = 1) <input type = "text" value="1" name = "RFallscale" /></p>
<p style="font-size:24px;">Hotel Options</p>
<p>Plot Hotels <input type="checkbox" name="PlotHot" checked></p>
<p>Minimum Danger Value (standard = 0) <input type = "text" value="0" name = "DangerVal" /></p>
<p><input type = "submit" value = "Submit" /></p></form>''' #literally just the html code for a form / index page

In [44]:
back ='''<form action="/form"><p><input type = "submit" value = "Change Values" /></p></form>'''# html code for back button

In [45]:
cssStuff = '''<title>The Map</title>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">
<link rel="stylesheet" href="https://fonts.googleapis.com/css?family=Lato">
<style>
body {
  font-family: "Lucida Console", monospace;
  background-color: black
}
p,h1 {
  color:white;
  font-size:16px;
}

</style>'''

In [ ]:
from flask import Flask,render_template,request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)   

@app.route('/', methods = ['POST', 'GET'])
def map():
  if request.method == 'GET': #on standard page view, values are default
        minpop = 1000000
        bfscale = 1
        bbscale = 1
        rfscale = 1
        rbscale = 1
        danger = 0
        plotbomb = True
        plotplant = True 
        plothot = True
        folium_map = mapmaker(plothot,plotplant,plotbomb, minpop, bfscale, bbscale,rfscale,rbscale,danger)#make map with set values
        return cssStuff + back + folium_map._repr_html_() #css, back button and map
  if request.method == 'POST':#after form submission, values are retrived from POST
        minpop = float(request.form['Minpop'])
        bfscale = float(request.form['BFallscale'])
        bbscale = float(request.form['BBoomscale'])
        rfscale = float(request.form['RFallscale'])
        rbscale = float(request.form['RBoomscale'])
        danger = float(request.form['DangerVal'])
        if (request.form.get('PlotBomb') == "on"):
           plotbomb = True
        else:
          plotbomb = False
        if (request.form.get('PlotPlant') == "on"):
           plotplant = True
        else:
          plotplant = False
        if (request.form.get('PlotHot') == "on"):
           plothot = True
        else:
          plothot = False
        folium_map = mapmaker(plothot,plotplant,plotbomb, minpop, bfscale, bbscale,rfscale,rbscale,danger)#make map with changed values
        return cssStuff + back + folium_map._repr_html_()#css, back button and map
  

@app.route('/form/')
def form():
    return cssStuff + formhtml #shows form as defined in formhtml variable

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e48b-35-245-249-164.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [28/Nov/2021 10:48:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 10:48:14] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [28/Nov/2021 10:48:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 10:48:51] "GET / HTTP/1.1" 200 -
